In [13]:
import os
os.getcwd()

'/home/onyxia/work'

In [14]:
os.chdir("..")
import utils.functions as fc
import utils.indicators as indic

In [35]:
from importlib import reload
fc = reload(fc)
indic = reload(indic)

In [5]:
ship_data_enriched = fc.create_ship_data_enriched()

In [6]:
ais = fc.read_ais_parquet()

In [7]:
ais.shape

(4406590, 6)

In [8]:
ais_enriched = fc.enrich_AIS_data(ais, ship_data_enriched)

In [18]:
#ais_enriched = fc.filter_cargo(ais_enriched)

In [9]:
ais_enriched.shape

(3326041, 125)

In [16]:
# function qui assigne le pays d'origine
ais_enriched2, origin_countries = indic.assign_country_flag(ais_enriched)

/home/onyxia/work/hackathon-un-2022/utils/indicators.py:14: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ais_enriched = pd.merge(ais_enriched, mmsi_number, on=["Digit"], how="left")


Classification of flag country
Russian Federation                                                                   387
Republic of Türkiye                                                                  158
Panama (Republic of)                                                                 120
Malta                                                                                 70
Liberia (Republic of)                                                                 54
Romania                                                                               47
Marshall Islands (Republic of the)                                                    47
Bulgaria (Republic of)                                                                36
Ukraine                                                                               26
Comoros (Union of the)                                                                21
Sierra Leone                                                                   

In [17]:
ais_enriched2["mmsi"].nunique()

1177

In [36]:
ais_enriched3, res_fuz_match2 = indic.fuzzy_match_destination_country(ais_enriched2)

Number of unique destination ports
4003
shape of ais
(3326041, 129)
shape of ais filtered with no na destination
(3040578, 129)
Fuzzy matching in process


/home/onyxia/work/hackathon-un-2022/utils/indicators.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["true_match"] = np.nan
/home/onyxia/work/hackathon-un-2022/utils/indicators.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_fuz_match2["matched_destination_country"] = np.nan


Classification of destination countries
Turkey           73
Russia           73
Germany          51
Romania          43
Ukraine          39
                 ..
Peru              1
Cote D'Ivoire     1
Madagascar        1
The Bahamas       1
Colombia          1
Name: matched_destination_country, Length: 86, dtype: int64
Percentage of boat with matched countries
0.655933762649494


In [37]:
res_fuz_match2.head()

,mmsi,matched_destination_country,matched_destination_port,ratio1,raw_port_destination
0,207008000,Georgia,POTI,100.000000,POTI
1,207037000,Bulgaria,VARNA,100.000000,VARNA
2,207038000,Denmark,ODENSE,83.333333,OPENSEA
3,207133000,Bulgaria,VARNA,100.000000,VARNA
4,207228000,Bulgaria,VARNA,100.000000,VARNA


In [38]:
res_fuz_match2.shape

(713, 5)